## Regression Analysis on the outputs of the B-Point detection algorithms

- The sampling rate of the empkins dataset 1000 Hz does not match the sampling rate of the guardian dataset 500 Hz. How to deal with that? 
    - Convert to milliseconds. Do it when building the train and test data
- Normalize B-Point samples acording to start and end point of heartbeat as part of data preprocessing?
    - Try both approaches (This kind of normalization does not make sense)
- Start without feature selection since the models should use all outputs generated by the algorithms
- How to impute nan values? If normalized between 0 and 1 just use mean?
    - drop them first
    - Background: Many algorithms don't handle NaN values
    - Check how many entries contain NaN
    - Use a SimpleImputer with e.g. mean or KNNImputer first to test the pipeline properly (https://scikit-learn.org/1.5/modules/impute.html)
- GroupKFold should be used for cross validation to ensure that a participant is not present in the train and testdata
- Splitting of the data:
    - use biopyskit and apply groupkfold. Mutliindex can remain in dataframe its important, that one hearbeat per row

### Setup and helper functions

In [ ]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer


#Feature Selection
from sklearn.feature_selection import SelectKBest, RFE

#Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# Cross-Validation
from sklearn.model_selection import GroupKFold

from biopsykit.classification.model_selection import SklearnPipelinePermuter

import matplotlib.pyplot as plt

%matplotlib widget
%load_ext autoreload
%autoreload 2

#### Specify whether the results should be saved or not

In [ ]:
save_results = False

### Load data

In [3]:
data_path = Path("../../results/train_test_data")
data_path

WindowsPath('../../results/train_test_data')

In [4]:
models_path = Path("../../results/models")

In [76]:
input_data = pd.read_csv(data_path.joinpath("combined_data.csv"))

In [77]:
X, y, groups = bp.classification.utils.prepare_df_sklearn(data=input_data, label_col="b_point_samplereference", subject_col="participant", print_summary=True)

KeyError: 'Requested level (b_point_samplereference) does not match index name (None)'

In [5]:
train_data = pd.read_csv(data_path.joinpath("train_data_all_algos.csv")).drop(columns=["Unnamed: 0"])
target_data = pd.read_csv(data_path.joinpath("target_data_all_algos.csv")).drop(columns=["Unnamed: 0"])

### Specify Estimator Combinations and Parameters for Hyperparameter Search

In [65]:
model_dict = {
    "scaler": {"StandardScaler": StandardScaler(), "MinMaxScaler": MinMaxScaler()},
    #"reduce_dim": {"SelectKBest": SelectKBest(), "RFE": RFE(SVC(kernel="linear", C=1))},
    "reduce_dim": {"SelectKBest": SelectKBest()},
    "clf": {
        #"KNeighborsRegressor": KNeighborsRegressor(),
        "RandomForestRegressor": RandomForestRegressor(),
        #"HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
    },
}

In [66]:
params_dict = {
    "StandardScaler": None,
    "MinMaxScaler": None,
    "SelectKBest": {"k": [2, 4, 6, 8, 10, "all"]},
    #"KNeighborsRegressor": {
    #    "n_neighbors": [8,9,10,11,12,13,14],
    #    "weights": ["uniform", "distance"],
    #    },
    "RandomForestRegressor": {
        #"n_estimators": [80, 100, 120],
        "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
        #"min_samples_split": [1, 2, 3, 4, 5],
        #"min_samples_leaf": [1, 2, 3, 4, 5],
        #"max_features": ["sqrt", "log2", None],
    },
    #"HistGradientBoostingRegressor": None,
}

In [67]:
#hyper_search_dict = {"DecisionTreeRegressor": {"search_method": "random", "n_iter":2}}

### Setup PipelinePermuter and Cross-Validations for Model Evaluation

In [68]:
pipeline_permuter = SklearnPipelinePermuter(
    model_dict=model_dict, param_dict=params_dict
)

In [69]:
X = train_data.drop(columns=["participant", "condition", "phase", "heartbeat_idreference"])
y = target_data.drop(columns=["participant", "condition", "phase", "heartbeat_idreference"])
groups = train_data["participant"]

In [70]:
outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=5)

pipeline_permuter.fit(X=X.values, y=y.values, outer_cv=outer_cv, inner_cv=inner_cv, scoring="neg_mean_absolute_error", groups=groups)

Pipeline Combinations:   0%|          | 0/2 [00:00<?, ?it/s]

### Running hyperparameter search for pipeline: (('scaler', 'StandardScaler'), ('reduce_dim', 'SelectKBest'), ('clf', 'RandomForestRegressor')) with 1 parameter grid(s):
Parameter grid #0 ({'search_method': 'grid'}): {'clf__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'], 'reduce_dim__k': [2, 4, 6, 8, 10, 'all']}


Outer CV:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


...

c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


Fitting 5 folds for each of 24 candidates, totalling 120 fits


c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sebas\Development\ResearchInternship\Code\pepbench\.venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\nested_cv.py:149: UserWarning: Cannot compute confusion matrix for regression tasks.
  warnings.warn("Cannot compute confusion matrix for regression tasks.")


### Display the resuslts of the pipeline permuter

To print the results I had to exclude the conf matrix in the Biopsykit function.  
Make sure to include it again afterwards

In [73]:
pipeline_permuter.metric_summary()

true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [[622.0], [1231.0], [3342.0], [4596.0], [5233....   
MinMaxScaler    SelectKBest         RandomForestRegressor  [[622.0], [1231.0], [3342.0], [4596.0], [5233....   

                                                                                           true_labels_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [[[622.0], [1231.0], [3342.0], [4596.0], [5233...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [[[622.0], [1231.0], [3342.0], [4596.0], [5233...   

                                                                                            predicted_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [624.22, 1273.84, 3340.91, 4641.41, 5282.65, 5...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [624.31, 1273.7, 3340.16, 4641.41, 5282.65, 58...   

                                                                                      predicted_labels_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [[624.22, 1273.84, 3340.91, 4641.41, 5282.65, ...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [[624.31, 1273.7, 3340.16, 4641.41, 5282.65, 5...   

                                                                                               train_indices  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                                                                         train_indices_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   

                                                                                                test_indices  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [1397, 1398, 1399, 1400, 1401, 1402, 1403, 140...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [1397, 1398, 1399, 1400, 1401, 1402, 1403, 140...   

                                                                                          test_indices_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                               
StandardScaler  SelectKBest         RandomForestRegressor  [[1397, 1398, 1399, 1400, 1401, 1402, 1403, 14...   
MinMaxScaler    SelectKBest         RandomForestRegressor  [[1397, 1398, 1399, 1400, 1401, 1402, 1403, 14...   

                                                           mean_test_neg_mean_absolute_error  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                               
StandardScaler  SelectKBest         RandomForestRegressor                          13.735736   
MinMaxScaler    SelectKBest         RandomForestRegressor                          13.723299   

                                                           std_test_neg_mean_absolute_error  \
pipeline_scaler pipeline_

In [74]:
pipeline_permuter.best_hyperparameter_pipeline()

C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\sklearn_pipeline_permuter.py:680: FutureWarning: ['param_clf__criterion', 'param_reduce_dim__k', 'params'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  .agg(["mean", "std"])


mean_test_neg_mean_absolute_error param_clf__criterion  \
outer_fold                                                           
0                                  -13.697996       absolute_error   
1                                  -13.957128       absolute_error   
2                                  -13.730262       absolute_error   
3                                  -14.304568       absolute_error   
4                                  -13.930380       absolute_error   

           param_reduce_dim__k  \
outer_fold                       
0                            4   
1                            4   
2                            4   
3                            4   
4                            4   

                                                       params  \
outer_fold                                                      
0           {'clf__criterion': 'absolute_error', 'reduce_d...   
1           {'clf__criterion': 'absolute_error', 'reduce_d...   
2           {'clf__criterion': 'absolute_error', 'reduce_d...   
3           {'clf__criterion': 'absolute_error', 'reduce_d...   
4           {'clf__criterion': 'absolute_error', 'reduce_d...   

            rank_test_neg_mean_absolute_error  \
outer_fold                                      
0                                           2   
1                                           2   
2                                           1   
3                                           1   
4                                           1   

            split0_test_neg_mean_absolute_error  \
outer_fold                                        
0                                    -13.787161   
1                                    -13.457105   
2                                    -13.451196   
3                                    -14.152876   
4                                    -13.213718   

            split1_test_neg_mean_absolute_error  \
outer_fold                                        
0                                    -14.243101   
1                                    -13.638498   
2                                    -14.598374   
3                                    -14.361676   
4                                    -12.648656   

            split2_test_neg_mean_absolute_error  \
outer_fold                                        
0                                    -13.618856   
1                                    -13.554444   
2                                    -13.303103   
3                                    -14.103827   
4                                    -14.886735   

            split3_test_neg_mean_absolute_error  \
outer_fold                                        
0                                    -13.241658   
1                                    -14.539383   
2                                    -13.980553   
3                                    -14.158255   
4                                    -15.116419   

            split4_test_neg_mean_absolute_error  \
outer_fold                                        
0                                    -13.599204   
1                                    -14.596208   
2                                    -13.318086   
3                                    -14.746205   
4                                    -13.786372   

            std_test_neg_mean_absolute_error  
outer_fold                                    
0                                   0.325379  
1                                   0.502224  
2                                   0.499298  
3                                   0.237916  
4                                   0.948521

### Save the results of the pipeline permuter to a pickle file

In [75]:
if save_results:
    pipeline_permuter.to_pickle(models_path.joinpath("Scaler_Feature_Elimination_Random_Forest.pkl"))